In [1]:
import os

In [2]:
%pwd

'/Users/rahulshelke/Documents/Data Science/Data Science Projects/End-to-end-Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rahulshelke/Documents/Data Science/Data Science Projects/End-to-end-Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:

        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [9]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

/Users/rahulshelke/Documents/Data Science/Data Science Projects/End-to-end-Text-Summarizer/summarizer-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class DataTransformation:

    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_dataset(self.config.data_path)
        # dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum-dataset"))

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

Generating train split: 14731 examples [00:00, 131457.40 examples/s]
Generating validation split: 818 examples [00:00, 78783.45 examples/s]
Generating test split: 819 examples [00:00, 98214.06 examples/s]
Map:   0%|          | 0/14731 [00:00<?, ? examples/s]/Users/rahulshelke/Documents/Data Science/Data Science Projects/End-to-end-Text-Summarizer/summarizer-venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 326285.62 examples/s]


### Error : TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [55]:
import pandas as pd

In [71]:
train = pd.read_csv("artifacts/data_ingestion/samsum-dataset/train.csv")
test = pd.read_csv("artifacts/data_ingestion/samsum-dataset/test.csv")
validation = pd.read_csv("artifacts/data_ingestion/samsum-dataset/validation.csv")

In [72]:
train[train['dialogue'].isna()]

,Unnamed: 0,id,dialogue,summary


In [73]:
train.isna().sum()

Unnamed: 0    0
id            0
dialogue      0
summary       0
dtype: int64

In [74]:
test.isna().sum()

id          0
dialogue    0
summary     0
dtype: int64

In [60]:
validation.isna().sum()

id          0
dialogue    0
summary     0
dtype: int64

- missing value creating error

In [52]:
train.dropna(axis=0, inplace=True)

In [69]:
if int(train.isnull().sum().sum()) > 0:
    print("yes")

yes


In [70]:
int(train.isnull().sum().sum())

1